In [3]:
from pathlib import Path
from simulator.engine.engine import SimulationResultConfig, SimulationEngine
from protocols.cora.v1.environments import HistoricalCoraEnvironment
from protocols.cora.v1.metrics import CoraMetrics
from protocols.cora.v1.strategies import CoraV1Strategy
from datetime import datetime, timedelta
from simulator.utilities.distributions import (
    TruncatedLogNormalDistribution,
    UniformDistribution,
    ParabolicDistribution,
    TruncatedNormalDistribution,
)

In [5]:
base_strategy_definition = {
    "utilization_parameter": 0.7,
    "loan_size_dist": {
        "type": "dist",
        "name": "truncated_lognormal",
        "params": {"lower": 1000, "upper": 100000, "mean": 3.5, "std": 7, "base": 10},
    },
    "loan_start_dist": {
        "type": "dist",
        "name": "uniform",
        "params": {"lower": 0, "upper": 1},
    },
    "loan_duration_dist": {
        "type": "dist",
        "name": "parabolic",
        "params": {"lower": 0, "upper": 1},
    },
    "ltv_dist": {
        "type": "dist",
        "name": "truncated_normal",
        "params": {"lower": 0.01, "upper": 0.99, "mean": 0.5, "std": 1.11},
    },
    "max_ltv": 0.4,
    "max_liquidity": 10000000,
    "initial_lending_amount": 10000000,
    "genesis_period_seconds": 86400,
    "running_period_seconds": 2592000,
    "fee_model": "black_scholes",
    "fee_model_update_params": {"lookback_days": 365, "volatility_factor": 1.0},
    "fee_model_update_interval_seconds": 86400,
}

utilization_parameters = [0.1, 0.3, 0.5, 0.7, 0.9, 1.1]
strategies = []
for utilization_parameter in utilization_parameters:
    new_strategy = base_strategy_definition.copy()
    new_strategy["utilization_parameter"] = utilization_parameter
    strategies.append(new_strategy)

In [6]:
results = []
for utilization_parameter, strategy in zip(utilization_parameters, strategies):

    strategy = CoraV1Strategy.from_dict(strategy)
    environment = HistoricalCoraEnvironment(symbol="ETH")
    metrics = CoraMetrics()
    config = SimulationResultConfig(
        results_folder="studies/002-utilization_rate/results",
        write_step_metrics=True,
        write_end_metrics=True,
        write_log=True,
        meta_log_level="INFO",
    )

    simulation_engine = SimulationEngine(
        strategy=strategy,
        environment=environment,
        metrics=metrics,
        name="utilizationparam_" + str(utilization_parameter),
        config=config,
    )

    start_date = datetime(2022, 1, 1)
    end_date = start_date + timedelta(days=31)
    time_step = timedelta(hours=1)

    print(f"Simulating with utilization parameter {utilization_parameter}")
    result = simulation_engine.run_simulation(
        start=start_date,
        end=end_date,
        step=time_step,
        random_seed=345,
    )
    results.append(result)

    print(f"Simulation complete => {simulation_engine._run_id}")


Simulating with utilization parameter 0.1
Simulation complete => 20220809T092846-utilizationparam_0.1-c706089-000000
Simulating with utilization parameter 0.3
Simulation complete => 20220809T092848-utilizationparam_0.3-3352589-000000
Simulating with utilization parameter 0.5
Simulation complete => 20220809T092851-utilizationparam_0.5-178e33f-000000
Simulating with utilization parameter 0.7
Simulation complete => 20220809T092853-utilizationparam_0.7-930d1f8-000000
Simulating with utilization parameter 0.9
Simulation complete => 20220809T092856-utilizationparam_0.9-25119cc-000000
Simulating with utilization parameter 1.1
Simulation complete => 20220809T092900-utilizationparam_1.1-6b74f6d-000000


In [11]:
results[0].end_metrics

{'pool_pnl': 0.6553140071132397,
 'ratio_loans_defaulted': [],
 'lending_fees': 0.6553140071132397,
 'hist-active_loans-ltv_0.0_0.1': 0,
 'hist-active_loans-ltv_0.1_0.2': 0,
 'hist-active_loans-ltv_0.2_0.3': 0,
 'hist-active_loans-ltv_0.3_0.4': 0,
 'hist-active_loans-ltv_0.4_0.5': 0,
 'hist-active_loans-ltv_0.5_0.6': 0,
 'hist-active_loans-ltv_0.6_0.7': 0,
 'hist-active_loans-ltv_0.7_0.8': 0,
 'hist-active_loans-ltv_0.8_0.9': 0,
 'hist-active_loans-ltv_0.9_1.0': 0,
 'hist-active_loans-duration_0.0_0.1': 0,
 'hist-active_loans-duration_0.1_0.2': 0,
 'hist-active_loans-duration_0.2_0.3': 0,
 'hist-active_loans-duration_0.3_0.4': 0,
 'hist-active_loans-duration_0.4_0.5': 0,
 'hist-active_loans-duration_0.5_0.6': 0,
 'hist-active_loans-duration_0.6_0.7': 0,
 'hist-active_loans-duration_0.7_0.8': 0,
 'hist-active_loans-duration_0.8_0.9': 0,
 'hist-active_loans-duration_0.9_1.0': 0,
 'hist-active_loans-start_0.0_0.1': 0,
 'hist-active_loans-start_0.1_0.2': 0,
 'hist-active_loans-start_0.2_0.